# Make the final SXDS catalogue

We want to take all the columns. We will save a catalogue for each patch which can be stacked to produce the final catalogues for serving publicly. The is_inner flags are used to remove the overlap regions. As far as possible we try to mimic the tables produced for HSC PDR2.

In [10]:
import json
import numpy as np
import lsst.daf.persistence as dafPersist

from astropy.table import Table
import astropy.units as u

In [2]:
hscBands = ['G', 'R', 'I', 'Z', 'Y']
vistaBands = ['Z', 'Y', 'J', 'H', 'Ks']
allBands = ['HSC-' +b for b in hscBands] + ['VISTA-' +b for b in vistaBands]

In [3]:
allBands

['HSC-G',
 'HSC-R',
 'HSC-I',
 'HSC-Z',
 'HSC-Y',
 'VISTA-Z',
 'VISTA-Y',
 'VISTA-J',
 'VISTA-H',
 'VISTA-Ks']

In [4]:
hscBands

['G', 'R', 'I', 'Z', 'Y', ['Z', 'Y', 'J', 'H', 'Ks']]

In [8]:
BUTLER_LOC = '/Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data'
BUTLER_LOC = '../../dmu4/dmu4_SXDS/data'
butler =  dafPersist.Butler(inputs='{}/rerun/coaddForcedPhot'.format(BUTLER_LOC))

In [7]:
full_patch_dict = json.load(open('../../dmu4/dmu4_SXDS/full_patch_dict.json', 'r'))

In [8]:
full_patch_dict

{'8282': ['0,8'],
 '8283': ['0,8',
  '1,7',
  '1,8',
  '2,7',
  '2,8',
  '3,7',
  '3,8',
  '4,7',
  '4,8',
  '5,7',
  '5,8',
  '6,7',
  '6,8',
  '7,7',
  '7,8',
  '8,8'],
 '8284': ['8,8'],
 '8523': ['0,1',
  '0,2',
  '0,3',
  '0,4',
  '0,5',
  '0,6',
  '0,7',
  '0,8',
  '1,1',
  '1,2',
  '1,3',
  '1,4',
  '1,5',
  '1,6',
  '1,7',
  '1,8',
  '2,1',
  '2,2',
  '2,3',
  '2,4',
  '2,5',
  '2,6',
  '2,7',
  '2,8',
  '3,1',
  '3,2',
  '3,3',
  '3,4',
  '3,5',
  '3,6',
  '3,7',
  '3,8',
  '4,1',
  '4,2',
  '4,3',
  '4,4',
  '4,5',
  '4,6',
  '4,7',
  '4,8',
  '5,2',
  '5,3',
  '5,4',
  '5,5',
  '5,6',
  '5,7',
  '5,8',
  '6,3',
  '6,4',
  '6,5',
  '6,6',
  '6,7',
  '6,8',
  '7,3',
  '7,4',
  '7,5',
  '7,6',
  '7,7',
  '7,8'],
 '8524': ['0,0',
  '0,1',
  '0,2',
  '0,3',
  '0,4',
  '0,5',
  '0,6',
  '0,7',
  '0,8',
  '1,0',
  '1,1',
  '1,2',
  '1,3',
  '1,4',
  '1,5',
  '1,6',
  '1,7',
  '1,8',
  '2,0',
  '2,1',
  '2,2',
  '2,3',
  '2,4',
  '2,5',
  '2,6',
  '2,7',
  '2,8',
  '3,0',
  '3,1',
  

In [ ]:
def makeCat(tract, patch, BUTLER_LOC):
    """make the final catalogue on a given patch for later stacking"""
    cat = Table()
    for band in allBands:
        
        CoaddCalexp = butler.get('deepCoadd_calexp',  {'filter': band, 'tract': tract, 'patch': patch})
        CoaddPhotoCalib = CoaddCalexp.getPhotoCalib()
        
        measSources = butler.get('deepCoadd_meas', {'filter': band, 'tract': tract, 'patch': patch})
        measSources = measSources.as_astropy()
        for c in measSources.colnames:
            if c != 'id':
                measSources[c].name = "{}_{}_{}".format(band,'meas', c)
                
        forcedSources = butler.get('deepCoadd_forced_src', {'filter': band, 'tract': tract, 'patch': patch})
        forcedCat = forcedSources.as_astropy()
        for c in forcedCat.colnames:
            if c != 'id':
                forcedCat[c].name = "{}_{}_{}".format(band,'forced', c)
        
        cat = join([cat, measCat],joinType='outer')
        cat = join([cat, forcedCat],joinType='outer')
                
        for c in cat.colnames:    
            if cat[c].unit = 'instFlux':
                if '_meas_' in c:
                    mags = CoaddPhotoCalib.instFluxToMagnitude(measSources, c)
                    flux = CoaddPhotoCalib.instFluxToNanojansky(measSources, c)
                elif '_forced_' in c:
                    mags = CoaddPhotoCalib.instFluxToMagnitude(forcedSources, c)
                    flux = CoaddPhotoCalib.instFluxToNanojansky(forcedSources, c)
                cat["{}_mag".format(c)] = mags[:,0]
                cat["{}_mag".format(c)].unit = u.mag
                cat["{}_magErr".format(c)] = mags[:,1]
                cat["{}_magErr".format(c)].unit = u.mag
                cat["{}_flux".format(c)] = flux[:,0]
                cat["{}_flux".format(c)].unit = u.nJy
                cat["{}_fluxErr".format(c)] = flux[:,1]
                cat["{}_fluxErr".format(c)].unit = u.nJy

    return cat

In [9]:
for tract in full_patch_dict:
    for patch in full_patch_dict[tract]:
        print(tract, patch)
        try:
            cat = makeCat(tract, patch, BUTLER_LOC)
            cat.write('./data/full_cat_{}_{}.fits'.format(tract,patch), overwrite=True)
        except:
            print('Failed.')

8282 0,8
8283 0,8
8283 1,7
8283 1,8
8283 2,7
8283 2,8
8283 3,7
8283 3,8
8283 4,7
8283 4,8
8283 5,7
8283 5,8
8283 6,7
8283 6,8
8283 7,7
8283 7,8
8283 8,8
8284 8,8
8523 0,1
8523 0,2
8523 0,3
8523 0,4
8523 0,5
8523 0,6
8523 0,7
8523 0,8
8523 1,1
8523 1,2
8523 1,3
8523 1,4
8523 1,5
8523 1,6
8523 1,7
8523 1,8
8523 2,1
8523 2,2
8523 2,3
8523 2,4
8523 2,5
8523 2,6
8523 2,7
8523 2,8
8523 3,1
8523 3,2
8523 3,3
8523 3,4
8523 3,5
8523 3,6
8523 3,7
8523 3,8
8523 4,1
8523 4,2
8523 4,3
8523 4,4
8523 4,5
8523 4,6
8523 4,7
8523 4,8
8523 5,2
8523 5,3
8523 5,4
8523 5,5
8523 5,6
8523 5,7
8523 5,8
8523 6,3
8523 6,4
8523 6,5
8523 6,6
8523 6,7
8523 6,8
8523 7,3
8523 7,4
8523 7,5
8523 7,6
8523 7,7
8523 7,8
8524 0,0
8524 0,1
8524 0,2
8524 0,3
8524 0,4
8524 0,5
8524 0,6
8524 0,7
8524 0,8
8524 1,0
8524 1,1
8524 1,2
8524 1,3
8524 1,4
8524 1,5
8524 1,6
8524 1,7
8524 1,8
8524 2,0
8524 2,1
8524 2,2
8524 2,3
8524 2,4
8524 2,5
8524 2,6
8524 2,7
8524 2,8
8524 3,0
8524 3,1
8524 3,2
8524 3,3
8524 3,4
8524 3,5
8524 3,6
8